In [1]:
import numpy as np

import cv2
from ultralytics import YOLO
from sort import Sort

In [2]:
def convertir(input_file, output_file, width=1024, height=768):
    # Captura el video de entrada
    cap = cv2.VideoCapture(input_file)

    # Verifica si se abrió el video correctamente
    if not cap.isOpened():
        print(f"No se pudo abrir el archivo: {input_file}")
        return

    # Obtener la resolución original
    original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    print(f"Resolución original: {original_width}x{original_height}")

    # Si la resolución es mayor que 1024x768, redimensionar
    if original_width > width or original_height > height:
        print("Redimensionando video...")
        # Obtener la tasa de cuadros y el codec del video original
        fps = cap.get(cv2.CAP_PROP_FPS)
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec para archivos MP4

        # Crear el objeto VideoWriter para el video de salida
        out = cv2.VideoWriter(output_file, fourcc, fps, (width, height))

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # Cambiar la resolución del frame
            resized_frame = cv2.resize(frame, (width, height))

            # Escribir el frame redimensionado en el video de salida
            out.write(resized_frame)

        # Liberar los objetos
        out.release()
        print(f"Video redimensionado guardado como: {output_file}")
    else:
        print("La resolución es adecuada. No se necesita redimensionar.")

    # Liberar el objeto de captura
    cap.release()

In [25]:
input_file = 'videos/Ejemplo1.mp4'  # Cambia esto por tu archivo de entrada
output_file = 'videos/video.mp4'  # Archivo de salida
   
convertir(input_file, output_file)

Resolución original: 1280x720
Redimensionando video...
Video redimensionado guardado como: videos/video.mp4


In [28]:
if __name__ == '__main__':
    cap = cv2.VideoCapture("videos/video.mp4")
    Y_minimo=450
    Y_maximo=500
    linea1 = [(0,Y_minimo),(1023,Y_minimo)]
    linea2 = [(0,Y_maximo),(1023,Y_maximo)]
    
              
    
    # Diccionario para contar las detecciones por clase
    clases_yolo = ["persona", "bicleta", "Auto", "Moto", "Avion", "Omnibus", "Tren", "Camion"]        
    deteccion_por_clase = {class_name: 0 for class_name in clases_yolo}
    objetos_contados = set()

    model = YOLO("yolov8s.pt")

    tracker = Sort()
    

    while cap.isOpened():
        status, frame = cap.read()

        if not status:
            break

        results = model(frame, stream=True)
        objetos_en_frame = set()  # Conjunto de IDs de objetos en el frame actual

        for res in results:
            filtered_indices = np.where((np.isin(res.boxes.cls.cpu().numpy(),[2,3,5,7])) & (res.boxes.conf.cpu().numpy()>0.4))[0] # Filtrar clases 2,3,5,7 => autos,motos,omnibus,camiones con mas de 40%
            boxes = res.boxes.xyxy.cpu().numpy()[filtered_indices].astype(int)
            class_indices = res.boxes.cls.cpu().numpy()[filtered_indices].astype(int)
            
            tracks = tracker.update(boxes)
            tracks = tracks.astype(int)
            
            cv2.line(frame,linea1[0],linea1[1],(255,0,0),3)
            cv2.line(frame,linea2[0],linea2[1],(255,0,0),3)
            
            
            #for xmin, ymin, xmax, ymax, track_id in tracks:
            for i, (xmin, ymin, xmax, ymax, track_id) in enumerate(tracks):
                # Obtener el índice de clase correspondiente y su nombre
                class_index = class_indices[i]
                class_name = clases_yolo[class_index]           
                xc,yc = int((xmin+xmax)/2),ymax
                
                print("Poscicion:")
                print(xc,yc,ymax)
                
                if (yc>Y_minimo and yc<Y_maximo):
                    if track_id not in objetos_contados:
                        # Incrementar el conteo de la clase detectada
                        deteccion_por_clase[class_name] += 1
                        # Agregar el ID al conjunto de objetos contados
                        objetos_contados.add(track_id)

                cv2.circle(img=frame, center=[xc,yc], radius=4, color=[0,255,0] , thickness=-1)
                cv2.rectangle(img=frame, pt1=(xmin, ymin), pt2=(xmax, ymax), color=(0, 255, 0), thickness=2)
                cv2.putText(img=frame, text=f"Id: {track_id}", org=(xmin, ymin-25), fontFace=cv2.FONT_HERSHEY_PLAIN, fontScale=1, color=(0, 255, 0), thickness=2)
                # Imprimir el nombre de la clase sobre el cuadro
                cv2.putText(img=frame, text=f"Clase: {class_name}", org=(xmin, ymin-10), fontFace=cv2.FONT_HERSHEY_PLAIN, fontScale=1, color=(0, 255, 0), thickness=2)
                cv2.putText(img=frame, text=f"Y : {yc}", org=(xmax, ymax-10), fontFace=cv2.FONT_HERSHEY_PLAIN, fontScale=1, color=(0, 255, 0), thickness=2)

      
        y_offset = 30  # Espaciado entre cada texto
        start_y = 50   # Ajusta esta coordenada para subir/bajar el texto en la pantalla
        for idx, (class_name, count) in enumerate(deteccion_por_clase.items()):
            if count > 0:  # Mostrar solo las clases que tienen detecciones
                text = f"{class_name}: {count}"
                # Colocar el texto en la parte superior izquierda de la pantalla
                cv2.putText(frame, text, (10, start_y + idx * y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        
        # frame = results[0].plot()

        cv2.imshow("frame", frame)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    
    cap.release()
    cv2.destroyAllWindows()


0: 480x640 6 cars, 2 trucks, 173.2ms
Poscicion:
11 330 330
Poscicion:
684 343 343
Poscicion:
589 336 336
Poscicion:
486 347 347
Poscicion:
664 460 460
Poscicion:
432 455 455
Poscicion:
329 477 477
Speed: 2.0ms preprocess, 173.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 7 cars, 2 trucks, 148.8ms
Poscicion:
431 363 363
Poscicion:
12 329 329
Poscicion:
589 336 336
Poscicion:
485 346 346
Poscicion:
661 457 457
Poscicion:
429 458 458
Poscicion:
324 481 481
Speed: 3.0ms preprocess, 148.8ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 8 cars, 2 trucks, 146.0ms
Poscicion:
429 362 362
Poscicion:
12 330 330
Poscicion:
588 335 335
Poscicion:
484 348 348
Poscicion:
657 453 453
Poscicion:
426 461 461
Poscicion:
320 485 485
Speed: 2.0ms preprocess, 146.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 8 cars, 1 truck, 149.0ms
Poscicion:
12 330 330
Poscicion:
587 335 335
Poscicion:
484 349 349
Poscici